In [1]:
from azure.cosmos import CosmosClient, PartitionKey, exceptions

In [2]:
from azure.cli.core import get_default_cli

def az_cli (args_str):
    args = args_str.split()
    cli = get_default_cli()
    cli.invoke(args)
    if cli.result.result:
        return cli.result.result
    elif cli.result.error:
        raise cli.result.error
    return True

In [3]:
RES_GROUP='seobot-resources-dev'
ACCT_NAME='seobot-dev-cosmos'

args = "cosmosdb show --resource-group {} --name {} --query documentEndpoint --output tsv".format(RES_GROUP, ACCT_NAME)
ACCOUNT_URI = az_cli(args)
args = "cosmosdb keys list --resource-group {} --name {} --query \"primaryMasterKey\" --output tsv".format(RES_GROUP, ACCT_NAME)
ACCOUNT_KEY = az_cli(args)

https://seobot-dev-cosmos.documents.azure.com:443/
NROrQMOzdTLXvUNMUFRGvys50Wxsj948HAn1HRyy4H6EF8GOGazfKZE3Nef7y0NRVcj0fNF9vvnC3Stw9I8iRQ==


In [4]:
client = CosmosClient(ACCOUNT_URI, {'masterKey': ACCOUNT_KEY})

In [5]:
database_name = 'seo-stats'
container_name = 'search_stats'
database = client.get_database_client(database_name)
print("Got database {}".format(database_name))
container = database.get_container_client(container_name)
print("Got container {}".format(container_name))

Got database seo-stats
Got container search_stats


In [6]:
## Run a query against the container to see number of documents
query = 'SELECT VALUE COUNT(1) FROM c'
result = list(container.query_items(query, enable_cross_partition_query=True))

print('Container with id \'{0}\' contains \'{1}\' items'.format(container.id, result[0]))

Container with id 'search_stats' contains '1182' items


In [7]:
query = 'SELECT c.searchDate, c.queryString, c.resultPage FROM c WHERE c.resultPage < 0'
result = list(container.query_items(query, enable_cross_partition_query=True))

In [8]:
import pandas as pd
df_cosmos = pd.DataFrame(result)
# Sample the top 10 results
df_cosmos.head(10)

,searchDate,queryString,resultPage
0,2021-04-08T00:03:59.686345Z,INJECTION THERAPY palm beach,-1
1,2021-04-08T00:18:22.4428872Z,INJECTION THERAPY palm beach,-1
2,2021-04-08T00:23:26.4380766Z,Medical clinic,-1
3,2021-04-08T00:24:23.3603504Z,Functional nutrition,-1
4,2021-04-08T00:25:26.282078Z,led light therapy palm beach,-1
5,2021-04-08T00:26:42.6456092Z,Functional nutrition,-1
6,2021-04-08T00:29:32.4457705Z,Fibromyalgia,-1
7,2021-04-08T00:30:26.6129885Z,Medical clinic,-1
8,2021-04-08T00:31:31.4310396Z,Back pain,-1
9,2021-04-08T00:32:22.4366959Z,INJECTION THERAPY palm beach,-1


## Mean Page Results Over All Time

In [9]:
df_rank = df_cosmos.groupby("queryString").mean().reset_index()
display(df_rank)

,queryString,resultPage
0,Acupressure,-1
1,Acupuncture,-1
2,Back pain,-1
3,Back pain relief,-1
4,Fibromyalgia,-1
5,Functional medicine,-1
6,Functional nutrition,-1
7,Gua Sha palm beach,-1
8,HYPNOBIRTHING palm beach,-1
9,Health assessment,-1
